<a href="https://colab.research.google.com/github/carlos-alves-one/-BlockChain-Explorer-Project/blob/main/blockchain_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Goldsmiths University of London
### MSc. Data Science and Artificial Intelligence
### Module: Blockchain Programming
### Author: Carlos Manuel De Oliveira Alves
### Student: cdeol003
### BlockChain Explorer Project

# Goal No.1

Calculate the transaction ID

## Step 1: Setup

Explain the setup process, including importing necessary libraries (e.g., hashlib for cryptographic hashing) and any initial configurations for the transaction.

In [48]:
import hashlib


## Step 2: Define Transaction Details
Outline how to define the transaction details, such as sender and recipient addresses, the amount, the fee, the nonce, and the digital signature. Mention that these details form the basis of the transaction and are crucial for its unique identification.

In [49]:
sender_addr      = "b66eeaa8059bd4d50c760beb93c2eb086ad91853"
recipient_addr   = "b8e6687270bade89f03178d0195dbde30be1287c"
sender_publickey = "3056301006072a8648ce3d020106052b8104000a03420004abb6abed08a9304fedaa394bd578fde6e28a3721ef25da5063fd3ddf"
amount = 5
fee    = 1
nonce  = 1
digital_signature = "39d22d5f4c0b28e15428c1a6c55818ade226a9f15968d3b9ef3d15253770f35e"


## Step 3: Define functions to Hash and calculate the Transaction ID


In [50]:
# SHA1 hash calculation
def calculate_sha1_hash(public_key):
    digest = hashes.Hash(hashes.SHA1())
    digest.update(public_key)
    return digest.finalize()

# Signature hash calculation
def calculate_signature_hash(recipient_hash, amount, fee, nonce):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(recipient_hash)
    digest.update(amount.to_bytes(8, byteorder='little', signed=False))
    digest.update(fee.to_bytes(8, byteorder='little', signed=False))
    digest.update(nonce.to_bytes(8, byteorder='little', signed=False))
    return digest.finalize()

# Transaction  ID calculation
def calculate_txid(sender_hash, recipient_hash, sender_public_key, amount, fee, nonce, signature):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(sender_hash)
    digest.update(recipient_hash)
    digest.update(sender_public_key)
    digest.update(amount.to_bytes(8, byteorder='little', signed=False))
    digest.update(fee.to_bytes(8, byteorder='little', signed=False))
    digest.update(nonce.to_bytes(8, byteorder='little', signed=False))
    digest.update(signature)
    return digest.finalize()


## Step 4: Calculates the Hashes and the Transaction ID


In [51]:
# Hash the sender address
sender_hash = calculate_sha1_hash(sender_addr.encode('utf-8'))

# Hash the recipient address
recipient_hash = calculate_sha1_hash(recipient_addr.encode('utf-8'))

# Calculate digital signature
digital_signature = calculate_signature_hash(recipient_hash, amount, fee, nonce)

# Encode the sender's public key from string to bytes using UTF-8 encoding
sender_publickey = sender_publickey.encode('utf-8')

# Generate the transaction ID
tx_id_bytes = calculate_txid(sender_hash, recipient_hash, sender_publickey, amount, fee, nonce, digital_signature)

# Convert the transaction ID from bytes to a hexadecimal string
tx_id = tx_id_bytes.hex()


## Step 5: Output the Transaction ID


In [52]:
print("The calculated transaction ID is:", tx_id)


The calculated transaction ID is: fe1f2d9e1651a97e0867ff8f38a52b56163a8c20e936d0f9fcbe29d01e7b1dcb


# Goal No.2

Verify the following transactions:
- Check the sender’s address,
- Check digital signature
- Check transaction ID

## Transaction No.1

In [53]:
sender_addr      = "3e6cfd7644cc96050c506d5246ea831998a98412"
recipient_addr   = "263b26367061e7c505b520a192c5c3d079f04808"
sender_publickey = "3056301006072a8648ce3d020106052b8104000a03420004b7348c4c2c 97fc8d0e1645fa868d32ca4de03f0772cb76bec66d535d977177896115445298b10cd378dd66afc6bff24a7827083251dd210303d0699f327a7134"
amount = 5
fee    = 1
nonce  = 1
digital_signature = "39d22d5f4c0b28e15428c1a6c55818ade226a9f15968d3b9ef3d15253770f35e"
transaction_id = "24639171a1a6fe688b410764c02d6f1a103fb5a16690ceef959e7647da8deb3d"


### Verify Sender's Address

#### Calculate SHA1 Hash of the Public Key

In [54]:
# Import the 'hashes' module from the 'cryptography.hazmat.primitives' package
from cryptography.hazmat.primitives import hashes

# Define a function to calculate SHA1 hash of a given public key
def calculate_sha1_hash(public_key):

    # Ensure public_key is in bytes
    if isinstance(public_key, str):

        # Encode string to bytes using UTF-8 encoding
        public_key = public_key.encode('utf-8')

    # Initialize a SHA1 hash object
    digest = hashes.Hash(hashes.SHA1())

    # Update the hash object with the public key data
    digest.update(public_key)

    # Finalize the hash calculation and return the hash value
    return digest.finalize()


#### Compare the Calculated Hash with the Known Address

This step compares the hash calculated from the sender's public key with a known address.

In [55]:
# Define a function to compare two addresses
def compare_addresses(calculated_hash, known_address):

    # Ensure both arguments are in the same format (bytes)

    # Check if the known address is a string
    if isinstance(known_address, str):

        # Encode the known address string to bytes using UTF-8 encoding
        known_address = known_address.encode('utf-8')


    # Compare the calculated hash with the known address and return the result
    return calculated_hash == known_address

# Call the compare_addresses function with the SHA1 hash of sender_publickey and recipient_addr
compare_addresses(calculate_sha1_hash(sender_publickey), recipient_addr)


False

The verification process returns `false` and indicates that the calculated value (derived from the sender's public key) does not match the provided address value.